In [1]:
import numpy as np
import numpy.matlib as ml
import numba
import matplotlib.pyplot as plt
from bcimodel import get_conditions, bciobjective
from test import simulate_data
N = 10000
pCommon, sigV, sigA, sigP = 0.27, 2, 8, 10
varV, varA, varP = sigV**2, sigA**2, sigP**2
possible_locations = np.array([-24,-12,0,12,24])
variance_conditions = [8]
conditions = get_conditions(possible_locations, variance_conditions)
data = simulate_data(possible_locations, variance_conditions)
pars = [pCommon, sigV, sigA, sigP, data, conditions, possible_locations, 100]
bciobjective(pars)

[7714, 2160, 126, 0, 0] 10000 [0.62745098 0.29411765 0.05882353 0.00980392 0.00980392] 1.0
-740.2853888075624
[2322, 5444, 2117, 117, 0] 10000 [0.15686275 0.59803922 0.2254902  0.00980392 0.00980392] 0.9999999999999999
-298.7977601115126
[124, 2080, 5524, 2166, 106] 10000 [0.00980392 0.14705882 0.73529412 0.09803922 0.00980392] 0.9999999999999999
-911.4363766887254
[0, 115, 2105, 5480, 2300] 10000 [0.00980392 0.00980392 0.5        0.47058824 0.00980392] 1.0
-6301.534754821347
[0, 1, 119, 2119, 7761] 10000 [0.00980392 0.00980392 0.04901961 0.74509804 0.18627451] 1.0
-8246.857969724355
[7689, 2184, 126, 1, 0] 10000 [0.14705882 0.81372549 0.01960784 0.00980392 0.00980392] 1.0
-9794.65132174379
[2257, 5486, 2110, 146, 1] 10000 [0.00970874 0.72815534 0.24271845 0.00970874 0.00970874] 1.0
-5320.732721547218
[126, 2078, 5424, 2259, 113] 10000 [0.00990099 0.25742574 0.63366337 0.08910891 0.00990099] 1.0
-874.0163737828552
[1, 128, 2125, 5525, 2221] 10000 [0.00980392 0.00980392 0.41176471 0.549

-107876.37302688695

In [3]:
N = 10000
pCommon, sigV, sigA, sigP = 0.27, 1, 1, 10
varV, varA, varP = sigV**2, sigA**2, sigP**2
vloc, aloc = 24, 24

In [6]:
from scipy.stats import multinomial
multinomial.logpmf(np.array([10, 10, 10, 10, 10]), 50, np.array([0.1, 0.1, 0.1, 0.35, 0.35]))

array(-17.11829119)

In [5]:
possible_locations = np.array([-24,-12,0,12,24])
variance_conditions = [8]
conditions = get_conditions(possible_locations, variance_conditions)
data = simulate_data(possible_locations, variance_conditions)
pars = [pCommon, sigV, sigA, sigP, data, conditions, possible_locations, 100]
bciobjective(pars)

[[8790, 1208, 2, 0, 0], [7707, 2156, 136, 1, 0]] 20000
[[8851, 1148, 1, 0, 0], [2237, 5511, 2123, 129, 0]] 20000
[[8839, 1159, 2, 0, 0], [134, 2130, 5574, 2050, 112]] 20000
[[8849, 1150, 1, 0, 0], [1, 115, 2156, 5497, 2231]] 20000
[[8873, 1124, 3, 0, 0], [0, 0, 133, 2166, 7701]] 20000
[[1126, 7705, 1168, 1, 0], [7679, 2192, 128, 1, 0]] 20000
[[1121, 7706, 1171, 2, 0], [2382, 5331, 2159, 127, 1]] 20000
[[1209, 7678, 1112, 1, 0], [124, 2152, 5485, 2142, 97]] 20000
[[1189, 7695, 1114, 2, 0], [1, 116, 2081, 5551, 2251]] 20000
[[1099, 7724, 1175, 2, 0], [0, 1, 120, 2175, 7704]] 20000
[[4, 1161, 7717, 1117, 1], [7727, 2142, 130, 1, 0]] 20000
[[2, 1119, 7734, 1142, 3], [2242, 5536, 2109, 111, 2]] 20000
[[1, 1152, 7685, 1162, 0], [121, 2136, 5463, 2174, 106]] 20000
[[2, 1131, 7746, 1120, 1], [1, 134, 2128, 5505, 2232]] 20000
[[2, 1139, 7728, 1131, 0], [0, 4, 144, 2133, 7719]] 20000
[[0, 1, 1134, 7727, 1138], [7709, 2176, 114, 1, 0]] 20000
[[0, 1, 1155, 7760, 1084], [2276, 5477, 2114, 132, 1]] 

nan

In [8]:
def clip(arr, roundup = 0.001): 
    arr = np.clip(arr, 0.001, np.inf)
    return arr

clip(np.array([1, 2, 3, 0]))

array([1.e+00, 2.e+00, 3.e+00, 1.e-03])

In [13]:
import numpy as np
def clip(arr, roundup = 0.001): # numpyify this function
    
    def clipper(x):
        for i in range(len(x)):
            if x[i] == 0:
                x[i] += roundup
                print(x[i])
            return x
        
    return np.apply_along_axis(clipper, 0, arr)

In [14]:
arr = clip(np.array([0, 0, 0, 23, 42, 51]))
print(arr)

0
[ 0  0  0 23 42 51]


In [113]:
def clip(arr, roundup = 0.001):
    arr = np.clip(arr, 0.001, np.inf)
    return arr

In [68]:
del arr

In [115]:
arr7 = np.array([0, 0, 0, 44, 55])
arr2 = clip(arr7)
arr2[3]

44.0

In [85]:
type(arr2)

NoneType

In [82]:
arr2

In [58]:
arr

array([ 0,  0,  0, 23, 42, 51])

In [124]:
x = np.array([5,5,5,5,6,6,6,6,66])
unique, counts = np.unique(x, return_counts = True)


(array([ 5,  6, 66]), array([4, 4, 1]))

In [129]:
def counter(arr, possible_locations):
    init_counts = np.zeros([5,])
    poss_arr = np.stack((possible_locations, init_counts), axis = 0)
    counts = np.asarray(np.unique(arr, return_counts = True))
    indices = np.where(np.in1d(poss_locs, counts))[0]
    # so we have the indices of the values we want in the counts array

    return counts

def counter2(binned, possible_locations): # base this on numpy too
    return [binned.count(bvc) for bvc in possible_locations]

counter(x)

array([4, 4, 1])

In [172]:
arr = np.array([4,4,4,4,4,4,4])
poss_locs, init_counts = np.array([1,2,3,4,5]), np.zeros([5,])
poss_arr = np.stack((poss_locs, init_counts), axis = 0)
counts = np.asarray(np.unique(arr, return_counts = True))
indices = np.where(np.in1d(poss_locs, counts))[0]
print(poss_arr)
print(counts)
print(indices)
# so then at index three of poss_arr[1]
# change to count value 

[[1. 2. 3. 4. 5.]
 [0. 0. 0. 0. 0.]]
[[4]
 [7]]
[3]


In [177]:
arr = np.array([1,2,3,4,5])
poss = np.array([1,2,3,4,5])
counter(arr, poss)

array([1, 1, 1, 1, 1])